In [ ]:
import numpy as np              # Numpy is the fundamental package for scientific computing in Python.
import matplotlib.pyplot as pl  # Matplotlib is a scientific plotting package.

# The statement below enforces the plots to be put into this notebook, instead of in their own windows.
%matplotlib inline
pl.rcParams.update({'font.size': 11})          # Set the standard font size of the plots to 11pt.
pl.rcParams.update({'figure.figsize': [13,5]}) # Set the standard figure size.

In [ ]:
nx = 256
ny = 128

Lx = 1e7
Ly = 5e6

u0 = 0.

g = 9.81
H = 100.
a = 6.37e6
lat = 45.
Omega = 7.292e-5
f0 = 2.*Omega*np.sin(np.deg2rad(lat))
beta = (2./a)*Omega*np.cos(np.deg2rad(lat))

dx = Lx / nx
dy = Ly / ny

x = np.arange(0, Lx, dx)
y = np.arange(0, Ly, dy)

xx, yy = np.meshgrid(x, y)

Psi0 = u0*yy

h_dist = 1.
L_dist = 2e6
x_sigma = L_dist/6.
h0 = np.exp(-( (xx-Lx/2.)**2 + (yy-Ly/2.)**2) / (2.*x_sigma**2))
h0 = h_dist*h0/np.max(h0)
Psi = Psi0 + g/f0*h0

u = -np.gradient(Psi, dy, axis=0)
v =  np.gradient(Psi, dx, axis=1)
h = f0/g*Psi

zeta = np.gradient(v, dx, axis=1) - np.gradient(u, dy, axis=0)

dPsidx = np.gradient(Psi, dx, axis=1)
dPsidy = np.gradient(Psi, dy, axis=0)
d2Psidx2 = np.gradient(dPsidx, dx, axis=1)
d2Psidy2 = np.gradient(dPsidy, dy, axis=0)

q0 = d2Psidx2 + d2Psidy2 + beta*yy - f0**2/(g*H)*Psi

In [ ]:
c0 = f0**2/(g*H)

# Modified wave numbers
k = 2.*(np.cos(2.*np.pi*np.arange(nx/2+1)/nx) - 1.) / dx**2
l = 2.*(np.cos(2.*np.pi*np.arange(ny    )/ny) - 1.) / dy**2

def invert_q(q_in):
    q_fft = np.fft.rfft2(q_in - beta*yy)
    Psi_fft = q_fft / (k[np.newaxis,:] + l[:,np.newaxis] - c0)
    return np.fft.irfft2(Psi_fft)

In [ ]:
Psi = invert_q(q0)
u = -np.gradient(Psi, dy, axis=0)
v =  np.gradient(Psi, dx, axis=1)
h = f0/g*Psi
pl.contourf(x, y, h, 31)
pl.colorbar()
pl.grid()
pl.tight_layout()

In [ ]:
dt = 1800.
runtime = 20 * 86400.
nsteps = round(runtime / dt)

q = q0.copy()
t = 0.
for n in range(nsteps):
    Psi = invert_q(q)
    u = -np.gradient(Psi, dy, axis=0)
    v =  np.gradient(Psi, dx, axis=1)
    dqdt = -u*np.gradient(q, dx, axis=1) \
           -v*np.gradient(q, dy, axis=0)
    q += dt*dqdt 
    t += dt

In [ ]:
Psi = invert_q(q)
u = -np.gradient(Psi, dy, axis=0)
v =  np.gradient(Psi, dx, axis=1)
h = f0/g*Psi
pl.contourf(x/1000, y/1000, h, 31)
pl.colorbar()
pl.streamplot(x/1000, y/1000, u, v, color='k')
pl.grid()
pl.title('h at t = {0:.0f} s, L$_R$ = {1:.0f} km'.format(t, (g*H)**.5/f0 / 1000))
pl.tight_layout()

In [ ]:
pl.figure()
pl.plot(x/1000, u[ny//2,:])
pl.plot(x/1000, v[ny//2,:])

In [ ]:
pl.figure()
pl.plot(x/1000, h0[ny//2,:], 'k:')
pl.plot(x/1000, h [ny//2,:], 'r-')